In [ ]:

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch


embeddings_path = '/content/downloadable_files/embeddings.pt'
faiss_index_path = '/content/downloadable_files/faiss_index.index'

print("Lade gespeicherte Embeddings und FAISS-Index...")
context_embeddings = torch.load(embeddings_path).numpy()
index = faiss.read_index(faiss_index_path)

if not isinstance(index, faiss.IndexFlatL2):
    raise ValueError("Der FAISS-Index wurde nicht korrekt geladen.")


print("Lade Sentence-Transformer-Modell...")
sentence_model = SentenceTransformer('all-mpnet-base-v2')


print("Lade GPT-3 Modell...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


contexts = []

def retrieve_relevant_contexts(question, index, contexts, top_k=5):
    """
    Sucht die relevantesten Kontexte im FAISS-Index basierend auf der Frage.
    """
    print("Berechne Embedding für die Frage...")
    # Berechne das Embedding der Frage
    question_embedding = sentence_model.encode([question], show_progress_bar=False)

    # Überprüfen, ob das Embedding als NumPy-Array vorliegt
    if not isinstance(question_embedding, np.ndarray):
        raise ValueError("Das Embedding der Frage ist nicht vom Typ numpy.ndarray.")

    print(f"Suche {top_k} relevante Kontexte im FAISS-Index...")
    distances, indices = index.search(np.array(question_embedding, dtype=np.float32), top_k)

    # Fehlerbehandlung, falls keine Kontexte gefunden werden
    if indices.shape[1] == 0:
        print("Keine relevanten Kontexte gefunden.")
        return []

    # Kontexte abrufen basierend auf den Indizes
    relevant_contexts = [contexts[idx] for idx in indices[0] if idx < len(contexts)]
    print(f"Gefundene relevante Kontexte: {relevant_contexts}")
    return relevant_contexts


def generate_answer(question, context):
    """
    Generiert eine Antwort unter Verwendung von GPT-3 basierend auf der Frage und dem Kontext.
    """

    input_text = f"Frage: {question}\nKontext: {context}\nAntwort:"

    n
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=200, num_return_sequences=1, no_repeat_ngram_size=2)

    # exctract answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


def answer_question(question):
    """
    Führt den gesamten QA-Prozess aus: Kontextsuche und Antwortgenerierung.
    """
    print("Frage erhalten:", question)
    relevant_contexts = retrieve_relevant_contexts(question, index, contexts)

    if not relevant_contexts:
        print("Keine relevanten Kontexte gefunden. Standardantwort wird generiert.")
        return "Entschuldigung, ich konnte keine passende Antwort finden."


    combined_context = " ".join(relevant_contexts[:3])  # Top-3-Kontext
    print("Erstelle Antwort basierend auf relevanten Kontexten...")
    answer = generate_answer(question, combined_context)
    return answer


question = "Was ist die Hauptstadt von Frankreich?"
answer = answer_question(question)
print("Antwort:", answer)


Lade gespeicherte Embeddings und FAISS-Index...


<ipython-input-14-abe2aa34af3d>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  context_embeddings = torch.load(embeddings_path).numpy()  # Lade die Embeddings


Lade Sentence-Transformer-Modell...
Lade GPT-3 Modell...
Frage erhalten: Was ist die Hauptstadt von Frankreich?
Berechne Embedding für die Frage...
Suche 5 relevante Kontexte im FAISS-Index...
Gefundene relevante Kontexte: []
Keine relevanten Kontexte gefunden. Standardantwort wird generiert.
Antwort: Entschuldigung, ich konnte keine passende Antwort finden.


In [ ]:
!pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)


In [ ]:
def find_top_k_contexts(question, model, index, contexts, k=3):


    question_embedding = model.encode([question], convert_to_tensor=True)


    distances, indices = index.search(question_embedding.cpu().numpy(), k=k)  # Top-k Ergebnisse

    relevant_contexts = [contexts[i] for i in indices[0]]
    return relevant_contexts


In [ ]:
def preprocess_context(context, max_length=512):

    return " ".join(context.split()[:max_length])


In [ ]:
def find_top_k_contexts(question, model, index, contexts, k=3):


    question_embedding = model.encode([question], convert_to_tensor=True)


    distances, indices = index.search(question_embedding.cpu().numpy(), k=k)  # Top-k Ergebnisse


    relevant_contexts = [contexts[i] for i in indices[0]]
    return relevant_contexts


In [ ]:
if __name__ == "__main__":
    print("Lade gespeicherte Embeddings und FAISS-Index...")


    embeddings = torch.load("embeddings.pt")
    index = faiss.read_index("faiss_index.index")


    print("Lade rc.wikipedia Validation-Dataset...")
    dataset = load_dataset("trivia_qa", "rc.wikipedia", split="validation")


    print("Extrahiere Kontexte...")
    contexts = []
    for example in dataset:
        entity_pages = example.get("entity_pages", {})
        wiki_context = entity_pages.get("wiki_context", "")

        if isinstance(wiki_context, list):
            wiki_context = " ".join(wiki_context)

        if wiki_context:
            contexts.append(wiki_context)

    print(f"Anzahl der Kontexte: {len(contexts)}")


    results = answer_first_100_questions(dataset, contexts, sentence_model, index, gen_model, gen_tokenizer)


    for i, result in enumerate(results):
        print(f"Frage {i+1}: {result['question']}")
        print(f"Antwort: {result['answer']}")
        print(f"Relevante Kontexte: {' '.join(result['contexts'])[:200]}...\n")


Lade gespeicherte Embeddings und FAISS-Index...


<ipython-input-9-7418683e7360>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("embeddings.pt")


FileNotFoundError: [Errno 2] No such file or directory: 'embeddings.pt'

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import torch
import numpy as np
import os

# 1. Laden und Kombinieren der Datensätze
print("Lade und kombiniere TriviaQA-Datensätze...")
combined_dataset = load_dataset("trivia_qa", "rc.wikipedia", split="train")
validation_dataset = load_dataset("trivia_qa", "rc.wikipedia", split="validation")


# 2. Modell für Sentence-Embeddings laden
print("Lade Sentence-Transformer-Modell...")
sentence_model = SentenceTransformer('all-mpnet-base-v2')

# 3. Kontexte extrahieren
print("Extrahiere Kontexte...")
contexts = []
for example in combined_dataset:
    entity_pages = example.get("entity_pages", {})
    wiki_context = entity_pages.get("wiki_context", "")

    if isinstance(wiki_context, list):
        wiki_context = " ".join(wiki_context)

    if wiki_context:
        contexts.append(wiki_context)

print(f"Anzahl der Kontexte: {len(contexts)}")

# 4. Embeddings der Kontexte generieren in Batches
batch_size = 32  # Wähle eine Batchgröße, die deinem System entspricht
context_embeddings = []

print("Generiere Embeddings für die Kontexte...")
for i in range(0, len(contexts), batch_size):
    batch = contexts[i:i + batch_size]
    embeddings = sentence_model.encode(batch, show_progress_bar=True)
    context_embeddings.append(embeddings)

# Verbinde alle Embeddings
context_embeddings = np.vstack(context_embeddings)

# 5. FAISS-Index erstellen
print("Erstelle FAISS-Index...")
dim = context_embeddings.shape[1]  # Dimension der Embeddings
index = faiss.IndexFlatL2(dim)  # L2-Distanz (euclidean distance) als Metrik
index.add(np.array(context_embeddings, dtype=np.float32))  # Embeddings in Index einfügen

# 6. Speicher- und Downloadpfade festlegen
embedding_file = "embeddings.pt"
faiss_index_file = "faiss_index.index"

# 6a. Embeddings speichern
print(f"Speichere Embeddings unter {embedding_file}...")
torch.save(torch.tensor(context_embeddings), embedding_file)

# 6b. FAISS-Index speichern
print(f"Speichere FAISS-Index unter {faiss_index_file}...")
faiss.write_index(index, faiss_index_file)

# 7. Bereitstellung für Download
print("Bereite Dateien für den Download vor...")
download_dir = "downloadable_files"
os.makedirs(download_dir, exist_ok=True)
os.rename(embedding_file, os.path.join(download_dir, embedding_file))
os.rename(faiss_index_file, os.path.join(download_dir, faiss_index_file))

print(f"Fertig! Dateien stehen im Verzeichnis '{download_dir}' bereit.")


Lade und kombiniere TriviaQA-Datensätze...


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
embedding_file = "embeddings.pt"
faiss_index_file = "faiss_index.index"

# 6a. Embeddings speichern
print(f"Speichere Embeddings unter {embedding_file}...")
torch.save(torch.tensor(context_embeddings), embedding_file)

# 6b. FAISS-Index speichern
print(f"Speichere FAISS-Index unter {faiss_index_file}...")
faiss.write_index(index, faiss_index_file)

# 7. Bereitstellung für Download
print("Bereite Dateien für den Download vor...")
download_dir = "downloadable_files"
os.makedirs(download_dir, exist_ok=True)
os.rename(embedding_file, os.path.join(download_dir, embedding_file))
os.rename(faiss_index_file, os.path.join(download_dir, faiss_index_file))

print(f"Fertig! Dateien stehen im Verzeichnis '{download_dir}' bereit.")


Speichere Embeddings unter embeddings.pt...
Speichere FAISS-Index unter faiss_index.index...
Bereite Dateien für den Download vor...
Fertig! Dateien stehen im Verzeichnis 'downloadable_files' bereit.


In [ ]:
!pip install datasets transformers sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import numpy as np
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import torch